# Web Scraping Homework - Mission to Mars

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import time

In [ ]:
# Setup config variables to enable Splinter interaction with browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

<strong> Hint:</strong> Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

In [ ]:
# Create dictionary to store news
scraped_data = {}

## NASA Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [ ]:
# Visit Nasa news url through splinter module
nasa_url = "https://redplanetscience.com" 
browser.visit(nasa_url)
# Wait for 5 seconds for error purpouses
time.sleep(5)

In [ ]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
# Get news title and news text by searching for appropriate div class 
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

In [ ]:
# Create dictionary to store data and save entries
scrape_nasa_news={"Title":news_title, "Paragraph":news_p}
scrape_nasa_news

In [ ]:
# Save scraped data as a new entry in the dictionary
scraped_data ["Title"] = news_title
scraped_data["Paragraph"] = news_p

## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://spaceimages-mars.com).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Find the image url to the full size .jpg image. Make sure to save a complete url string for this image.

In [ ]:
mars_url = "https://spaceimages-mars.com" 
browser.visit( mars_url)
image_html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs( image_html, "html.parser")

In [ ]:
featured_image = soup.find_all("img", class_ = "headerimage fade-in")[0]["src"]
featured_image_url = mars_url + "/" + featured_image
print(featured_image_url)

In [ ]:
# Create dictionary to store data and save entries
jpl = {"ImageURL":featured_image_url}
jpl

In [ ]:
# Save scraped data as a new entry in the dictionary
scraped_data["ImageURL"] = featured_image_url

In [ ]:
browser.quit()

## Mars Facts

Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com/) and use Pandas to scrape the table containing facts about the planet.
Use Pandas to convert the data to a HTML table string.

In [ ]:
facts_url = "https://galaxyfacts-mars.com/"
facts_data = pd.read_html(facts_url)
facts_data

In [ ]:
facts_data[0]

In [ ]:
facts_data[1]

In [ ]:
mars_facts = facts_data[1]
mars_facts.rename(columns={0: "Description",1: "Mars"}, inplace=True)
mars_facts = mars_facts.set_index("Description")
mars_facts

In [ ]:
facts_table = mars_facts.to_html(index=False)
facts_table

In [ ]:
# Check out table
# facts_table.replace("\n", "")
# print(facts_table)

In [ ]:
# Create dictionary to store data and save entries
scraped_data["TableHTML"] = mars_facts.to_html()

## Mars Hemispheres:

Visit the USGS Astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

<strong>Example:</strong> hemisphere_image_urls = [ {"title": "Valles Marineris Hemisphere", "img_url": "..."}, {"title": "Cerberus Hemisphere", "img_url": "..."}, {"title": "Schiaparelli Hemisphere", "img_url": "..."}, {"title": "Syrtis Major Hemisphere", "img_url": "..."}, ]

In [ ]:
# Visit hemisphere url through splinter module
#Mars Hemispheres
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

hem_url = 'https://marshemispheres.com/'
browser.visit(hem_url)

In [ ]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
items = soup.find_all('div', class_='item')

In [ ]:
hemi_urls = []
hemi_title = []

# Create list of dictionaries for each hemisphere and append the dict 
# with an image URL string and title.
for item in items:
    hemi_urls.append( hem_url + item.find('a')['href'])
    hemi_title.append( item.find('h3').text.strip())

print( hemi_urls)
hemi_title

In [ ]:
hemi_img_urls = []

for url in hemi_urls:
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Find image urls and append to list
    source_url = hem_url + soup.find('img',class_='wide-image')['src']
    hemi_img_urls.append( source_url)
    
hemi_img_urls

In [ ]:
hemisphere_urls = []

for i in range(len(hemi_title)):
    hemisphere_urls.append({"title":hemi_title[i], "url":hemi_img_urls[i]})

hemisphere_urls

In [ ]:
browser.quit()

In [ ]:
# Dictionary of all Mars info scraped
mars_dict ={"Title": news_title, "Paragraph": news_p, "ImageURL": featured_image_url, 
            "Table": facts_table, "ListImages": hemisphere_urls}
mars_dict

## MongoDB and Flask Application

- Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.
- Start by converting your Jupyter notebook into a Python script called scrape_mars.py with a function called scrape that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.
- Next, create a route called /scrape that will import your scrape_mars.py script and call your scrape function.
- Store the return value in Mongo as a Python dictionary.
- Create a root route / that will query your Mongo database and pass the mars data into an HTML template to display the data.
- Create a template HTML file called index.html that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

In [ ]:
# Use flask_pymongo to set up mongo connection
from pymongo import MongoClient
conn =  "mongodb://localhost:27017/mars_mission_scraping"
client =  pymongo.MongoClient(conn)

In [ ]:
# Get collection and drop existing data for this application
db = client.mars_mission_scraping
db.mars_data.drop()

In [ ]:
db.mars_data.insert_many([scraped_data])

In [ ]:
query_result = (db.mars_data.find())
query_result